# Demo project - Wine quality prediction

## Contents:
* [Import packages](#first-bullet)
* [Load Data](#second-bullet)
* [Exploratory data analysis](#third-bullet)
* [Prepare dataset for training model](#forth-bullet)
* [Build a baseline model](#fifth-bullet)
* [Experiment with a new model](#sixth-bullet)
* [Predict](#seventh-bullet)

## Import packages <a class="anchor" id="first-bullet"></a>

We will need to install and import packages as we develop our notebook. We've created a couple of starter cells for you but you will need to add more as you work through the notebook.

In [ ]:
!pip install s3fs
# Install more modules that you need here
!pip install seaborn

In [ ]:
import pandas
# Import more modules and classes that you need here - REMEMBER TO RERUN THE CELL AFTER MODIFYING!
import os
import seaborn

## Load Data <a class="anchor" id="second-bullet"></a>

You have access to a Minio-based S3 storage where your datasets are available, and where you will eventually push models. This storage is defined using a 'Data Connection' in your Data Science Project. You can access this data connection using environment variables. Run the following shell block to determine the environment variable names:

In [ ]:
!env | grep AWS

You will need to assign these to Python variables to be able to use them in code blocks. We've started you off with some code below, but you'll also need variables set for the endpoint and bucket. Remember to import modules as needed in the import block at the top of the Notebook and re-run the cell below again after importing any modules.

In [ ]:
AWS_ACCESS_KEY_ID = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = os.environ['AWS_SECRET_ACCESS_KEY']
# Add variable assignments for AWS_S3_ENDPOINT and AWS_S3_Bucket below.
AWS_S3_ENDPOINT = os.environ['AWS_S3_ENDPOINT']
AWS_S3_BUCKET = os.environ['AWS_S3_BUCKET']


Check that your variables have been correctly set:

In [ ]:
print("AWS_ACCESS_KEY is " + AWS_ACCESS_KEY_ID)
print("AWS_SECRET_ACCESS_KEY is " + AWS_SECRET_ACCESS_KEY)
print("AWS_S3_ENDPOINT is " + AWS_S3_ENDPOINT)
print("AWS_S3_BUCKET is " + AWS_S3_BUCKET)


## Exploratory data analysis <a class="anchor" id="third-bullet"></a>
Have a look in the Minio UI and you will see that you have two datafiles in your bucket, called winequality-red.csv and winequality-white.csv. Let's set up some code to pull these from the storage into memory so that we can start some statistical exploration and visualisation. We will use the Pandas module to do this.

First we define a function, read_data() which uses a pandas method to read CSVs directly from S3 storage. Note how we pass our S3 credentials to the method. Because this is a function definition it won't actually do anything when you execute the code cell. 

In [ ]:
def read_data(datasrc):
    data = pandas.read_csv(
        "s3://" + AWS_S3_BUCKET + "/" + datasrc, sep=';',
        storage_options={
            "key": AWS_ACCESS_KEY_ID,
            "secret": AWS_SECRET_ACCESS_KEY,
            "endpoint_url": AWS_S3_ENDPOINT,
        }
    )
    return data

Let's try reading our two CSV files into memory now.

In [ ]:
white_wine = read_data('winequality-white.csv')
red_wine = read_data('winequality-red.csv')

Let's have a look at our white wine data:

In [ ]:
white_wine.head(5)


In [ ]:
# add a command in this cell to inspect our red wine data
red_wine.head(5)

We would like to run analysis on both our red and white wine datasets simultaneously, so it makes sense to merge these two datasets into one. But how will we then tell the difference between our red and white wines? Well, we simply add another feature - the feature is calles 'is_red' and is essentially a Boolean indicating whether the wine is red, or 'not red' i.e. white.

(Extra credit for anyone who can point out what might be problematic about this approach!)

Let's define a function to definte our additional feature in each dataset, and then concatenate them.

In [ ]:
def transformdata(red_wine,white_wine):
    red_wine['is_red'] = 1
    white_wine['is_red'] = 0
    data = pandas.concat([red_wine, white_wine], axis=0)
    # lets get rid of those annoying spaces in our column names
    data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
    return data

In [ ]:
Now, invoke your method and show the first 5 lines of the merged data below:

In [ ]:
# Write your code here
# data = insert your method call here
data = transformdata(red_wine,white_wine)
data.head(5)

SLIDES TO DISCUSS EXPLORATORY STATS COVERING:
- visualisation basics
- mean, median, deviation, skew
- quartiles and outliers
- correlation


Let's visualise our data using the seaborn module. Remember you may to install and/or import the module in the block at the beginning of the notebook (and re-run). Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics.You can read more about it here: https://seaborn.pydata.org/

This will plot a histogram of the quality of our wine. Experiement with plotting different features of the dataset, e.g. alcohol content, pH etc.

In [ ]:
seaborn.displot(data.quality, kde=False)

Let's simplify things by converting quality from a 1-10 scale to a simple boolean. A wine is either of high quality, or it is not. This quality feature will be our output feature when we run an inference model.

In [ ]:
def settarget(data):
    high_quality = (data.quality >= 7).astype(int)
    data.quality = high_quality
    return data

data = settarget(data)
data.tail(5)

In [ ]:
import seaborn as sns
sns.displot(data.quality, kde=False)

In [ ]:
## median, upper and lower quartile, IQR
## histogram for distribution

In [ ]:
dims = (3, 4)

f, axes = plt.subplots(dims[0], dims[1], figsize=(25, 15))
axis_i, axis_j = 0, 0
for col in data.columns:
  if col == 'is_red' or col == 'quality':
    continue # Box plots cannot be used on indicator variables
  sns.boxplot(x=data['quality'], y=data[col], ax=axes[axis_i, axis_j])
  axis_j += 1
  if axis_j == dims[1]:
    axis_i += 1
    axis_j = 0

Check missing value

In [ ]:
## scenarios for missing data - decision for the missing data
## if alcohol is not an indicator, delete that record

## what are we going to do with the outliers? are they real outliers?

In [ ]:
data.isna().any()

## Prepare dataset for training model <a class="anchor" id="forth-bullet"></a>
Split the input data into 3 sets:

- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [ ]:
def get_trainingdata(data):
    X = data.drop(["quality"], axis=1)
    y = data.quality

    # Split out the training data
    X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.6, random_state=123)

    # Split the remaining data equally into validation and test
    X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=123)
    return (X_train,X_val,X_test,y_train,y_val,y_test)

In [ ]:
(X_train,X_val,X_test,y_train,y_val,y_test) = get_trainingdata(data)

## Build a baseline model (random forest classifier) <a class="anchor" id="fifth-bullet"></a>
Build a simple classifier using scikit-learn. Use MLflow to keep track of the model accuracy. You can read about Classification - ROC and AUC here if you want 
https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc

Enable MLflow autologging

In [ ]:
experiment_name = "WineQuality"

In [ ]:
# check if experiment name already exists
mlflow.set_tracking_uri("http://mlflow:5500")
mlflow.set_experiment(experiment_name)

# enable autologging
mlflow.sklearn.autolog(log_input_examples=True)

In [ ]:
def log_featureimportance(model):
    tmpdir = tempfile.mkdtemp()
    filepath = os.path.join(tmpdir, 'feature_importance.json')
    feature_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns.tolist(), columns=['importance'])
    feature_importances.sort_values('importance', ascending=False).to_json(filepath)
    mlflow.log_artifact(filepath)
    return

Train random forest

In [ ]:
class SklearnModelWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        self.model = model

    def predict(self, context, model_input):
        return self.model.predict_proba(model_input)[:,1]

def train_randomforest(X_train,y_train,X_test,y_test):

    with mlflow.start_run(run_name='untuned_random_forest'):
        n_estimators = 10
        model = RandomForestClassifier(n_estimators=n_estimators, random_state=np.random.RandomState(123))
        model.fit(X_train, y_train)

        predictions_test = model.predict_proba(X_test)[:,1]
        auc_score = roc_auc_score(y_test, predictions_test)
        mlflow.log_param('n_estimators', n_estimators) #specify the interested parameter/metric
        mlflow.log_metric('auc', auc_score)
        wrappedModel = SklearnModelWrapper(model)

        signature = infer_signature(X_train, wrappedModel.predict(None, X_train))

        conda_env = _mlflow_conda_env(
            additional_conda_deps=None,
            additional_pip_deps=["cloudpickle=={}".format(cloudpickle.__version__), "scikit-learn=={}".format(sklearn.__version__)],
            additional_conda_channels=None,
            )
        mlflow.pyfunc.log_model("random_forest_model", python_model=wrappedModel, conda_env=conda_env, signature=signature)
        log_featureimportance(model)
        return model

In [ ]:
model = train_randomforest(X_train,y_train,X_test,y_test)

In [ ]:
# Sanity-check: This should match the AUC logged by MLflow
print(f'AUC: {roc_auc_score(y_test, model.predict_proba(X_test)[:,1])}')

In [ ]:
# Sanity-check: This should match the feature importance logged by MLflow
feature_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)

## Experiment with a new model (xgboost) <a class="anchor" id="sixth-bullet"></a>
Use the xgboost library to train a more accurate model. Run hyperparameter tuning to train multiple models. As before, the code tracks the performance of each parameter configuration with MLflow.

In [ ]:
search_space = {
  'max_depth': scope.int(hp.quniform('max_depth', 50, 100, 10)),
  'learning_rate': hp.loguniform('learning_rate', -3, 0),
  'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
  'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
  'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
  'objective': 'binary:logistic',
  'seed': 123, # Set a seed for deterministic training
}

def train_model(params):

  mlflow.xgboost.autolog()
  with mlflow.start_run(nested=True):
    train = xgb.DMatrix(data=X_train, label=y_train)
    validation = xgb.DMatrix(data=X_val, label=y_val)

    booster = xgb.train(params=params, dtrain=train, num_boost_round=100,\
                        evals=[(validation, "validation")], early_stopping_rounds=50)
    validation_predictions = booster.predict(validation)
    auc_score = roc_auc_score(y_val, validation_predictions)
    mlflow.log_metric('auc', auc_score) #specify the interested parameter/metric

    signature = infer_signature(X_train, booster.predict(train))
    mlflow.xgboost.log_model(booster, "model", signature=signature)

    return {'status': STATUS_OK, 'loss': -1*auc_score, 'booster': booster.attributes()}

with mlflow.start_run(run_name='xgboost_models'):
  best_params = fmin(
    fn=train_model,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
  )

In [ ]:
best_run = mlflow.search_runs(order_by=['metrics.auc DESC']).iloc[0]
best_run_id = best_run["run_id"]
print(f'AUC of Best Run: {best_run["metrics.auc"]}')

In [ ]:
best_run_id

## Predict <a class="anchor" id="seventh-bullet"></a>

In [ ]:
# model = mlflow.pyfunc.load_model(f"models:/TestModelD/production")
model = mlflow.pyfunc.load_model("runs:/" + best_run_id + "/model")

test_predictions = model.predict(X_test)
print(f'AUC: {roc_auc_score(y_test, test_predictions)}')

In [ ]:
from sklearn.metrics import classification_report

class_labels = ['white wine', 'red wine']
test_predictions = np.where(test_predictions>0.5, 1, 0)
print(classification_report(y_test, test_predictions, target_names=class_labels))

In [ ]:
cm = confusion_matrix(y_test, test_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)

disp.plot()
plt.show()

In [ ]:
# register the best model
new_model_version = mlflow.register_model(f"runs:/{best_run_id}/model", "WineQuality")

In [ ]:
# # Promote the new model version to Production
# client.transition_model_version_stage(
#   name="TestModelD",
#   version=new_model_version.version,
#   stage="Production"
# )

In [ ]:
# # clean up models
# from mlflow.tracking import MlflowClient
# client = MlflowClient()
# client.delete_registered_model(name="winequality")